In [1]:
import pandas as pd
pd.read_csv('/root/kostya/mcts/mcts-llm/src/results/AIME_2022_llama_3_8b.csv')
# question	correct_answer	mcts_answer	best_q_value
# 0	Find the three-digit positive integer $\underl...	227	## Solution\n\nLet's start by rearranging the ...	-33.542969

,question,correct_answer,mcts_answer,best_q_value
0,Find the three-digit positive integer $\underl...,227,## Solution\n\nLet's start by rearranging the ...,-33.542969
1,"In isosceles trapezoid $ABCD,$ parallel bases ...",242,## $\frac{333\sqrt{109}}{50}$,-33.277344
2,Let $w = \dfrac{\sqrt{3} + i}{2}$ and $z = \df...,834,## Solution\n\nWe can simplify the equation $i...,-10.314453
3,A straight river that is $264$ meters wide flo...,550,1. Given the river's width (264 meters) and t...,-20.513672
4,Find the number of ordered pairs of integers $...,228,"To solve this problem, we'll consider the cons...",-14.876953
5,"Let $a,b,c,d,e,f,g,h,i$ be distinct integers f...",289,1. **Assign Values to Minimize the Numerator:*...,17.137695
6,Equilateral triangle $\triangle ABC$ is inscri...,378,$$\ns = 8 = \sqrt{64} - \sqrt{0}.\n$$\n\nHowev...,-43.828125
7,"Ellina has twelve blocks, two each of red ( $\...",247,$\frac{3}{8}$,-31.375000
8,"Three spheres with radii $11,$ $13,$ and $19$ ...",756,"To solve for $AC^2$, we can use the Pythagorea...",28.308594
9,Let $ABCD$ be a parallelogram with $\angle BAD...,150,## Step 1: Draw the perpendicular from P to AC...,-13.371094


In [2]:
from src.mcts_llm.llm import openai_chat_completion
from src.mcts_llm.prompt_configs import llama_3_70b_prompt_config
def validate_answer(problem: str, correct_solution: str, given_answer: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "Evaluate the given answer against the correct solution. Output HIGH if the answer and solution match closely, MID if they are somewhat close, and LOW if they are not close at all. Consider both numerical accuracy and reasoning quality in your evaluation.",
        },
        {
            "role": "user",
            "content": "\n\n".join(
                [
                    f"<problem>\n{problem}\n</problem>",
                    f"<correct_solution>\n{correct_solution}\n</correct_solution>",
                    f"<given_answer>\n{given_answer}\n</given_answer>",
                ]
            ),
        },
    ]

    response = openai_chat_completion(
        messages=messages,
        model=llama_3_70b_prompt_config.model,
        base_url=llama_3_70b_prompt_config.base_url,
        max_tokens=200,
    )

    content = response.choices[0].message.content
    if content is not None:
        if "HIGH" in content.upper():
            return "HIGH"
        elif "MID" in content.upper():
            return "MID"
        elif "LOW" in content.upper():
            return "LOW"

    return "INVALID"

# Example usage
df = pd.read_csv('/root/kostya/mcts/mcts-llm/src/results/AIME_2022_llama_3_8b.csv')
df['validation_result'] = df.apply(lambda row: validate_answer(row['question'], row['correct_answer'], row['mcts_answer']), axis=1)
print(df[['question', 'correct_answer', 'mcts_answer', 'validation_result']])

                                             question  \
0   Find the three-digit positive integer $\underl...   
1   In isosceles trapezoid $ABCD,$ parallel bases ...   
2   Let $w = \dfrac{\sqrt{3} + i}{2}$ and $z = \df...   
3   A straight river that is $264$ meters wide flo...   
4   Find the number of ordered pairs of integers $...   
5   Let $a,b,c,d,e,f,g,h,i$ be distinct integers f...   
6   Equilateral triangle $\triangle ABC$ is inscri...   
7   Ellina has twelve blocks, two each of red ( $\...   
8   Three spheres with radii $11,$ $13,$ and $19$ ...   
9   Let $ABCD$ be a parallelogram with $\angle BAD...   
10  For any finite set $X,$ let $|X|$ denote the n...   
11  Let $S$ be the set of all rational numbers tha...   
12  Given $\triangle ABC$ and a point $P$ on one o...   
13  Let $x,$ $y,$ and $z$ be positive real numbers...   
14  Adults made up $\frac5{12}$ of the crowd of pe...   
15  Azar, Carl, Jon, and Sergey are the four playe...   
16  A right square pyramid with

In [4]:
df[['validation_result']].value_counts()

validation_result
LOW                  24
HIGH                  2
INVALID               2
MID                   1
Name: count, dtype: int64

In [4]:
def load_aime(year: int | None = None) -> pd.DataFrame:
    df = pd.read_csv("datasets/AIME_Dataset_1983_2024.csv")
    df.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)
    if year is not None:
        df = df[df["Year"] == year]

    return df

In [9]:
load_aime(2022).iloc[3]['question']

'Let $w = \\dfrac{\\sqrt{3} + i}{2}$ and $z = \\dfrac{-1 + i\\sqrt{3}}{2},$ where $i = \\sqrt{-1}.$ Find the number of ordered pairs $(r,s)$ of positive integers not exceeding $100$ that satisfy the equation $i \\cdot w^r = z^s.$'